## Objectives
* Solve some real world questions related to IPL 2018 using numpy and pandas functions and structure

### What do you need to solve the questions?
* Understanding of functions belonging to pandas library : 
  * How to read data from a URL
  * What is a dataframe? What is a Series?
  * How to read rows and columns from a dataframe
  * How to group rows in a dataframe (just like the groupby clause in SQL)
  * How to perform aggregations (sum/min/max/average) on a column/group of columns in a dataframe
  * How to create calculated columns in a dataframe?
  
### Some good starting points : 
* [10 minutes to Pandas](http://pandas.pydata.org/pandas-docs/stable/10min.html)
* If you like learning from videos, [here](https://www.youtube.com/playlist?list=PL5-da3qGB5ICCsgW1MxlZ0Hq8LL5U3u9y) is an engaging series

List down your understanding of the dataset

The data set describes the performance various batamen belonging to different teams in various matches and in different innings order at different venues. Also gives description about the number of boundaries, dismissal mode, number of balls faced, matche's date.

In [2]:
import pandas as pd
import numpy as np

In [13]:
#what is the maximum score of IPL 2018 and who scored it?
df = pd.read_csv('iplfinal.csv',sep='|')

df.head()


,match_description,match_date,match_venue,match_location,match_result,team_name,innings_order,batsman_name,dismissal_mode,runs,balls,fours,sixes
0,"Mumbai Indians vs Chennai Super Kings, 1st Mat...",2018-04-07,"Wankhede Stadium,",Mumbai,Chennai Super Kings won by 1 wkt,Mumbai Indians Innings,innings_1,Rohit Sharma (c),c Rayudu b Watson,15,18,1,1
1,"Mumbai Indians vs Chennai Super Kings, 1st Mat...",2018-04-07,"Wankhede Stadium,",Mumbai,Chennai Super Kings won by 1 wkt,Mumbai Indians Innings,innings_1,Evin Lewis,lbw b D Chahar,0,2,0,0
2,"Mumbai Indians vs Chennai Super Kings, 1st Mat...",2018-04-07,"Wankhede Stadium,",Mumbai,Chennai Super Kings won by 1 wkt,Mumbai Indians Innings,innings_1,Ishan Kishan (wk),c Mark Wood b Tahir,40,29,4,1
3,"Mumbai Indians vs Chennai Super Kings, 1st Mat...",2018-04-07,"Wankhede Stadium,",Mumbai,Chennai Super Kings won by 1 wkt,Mumbai Indians Innings,innings_1,Suryakumar Yadav,c Harbhajan b Watson,43,29,6,1
4,"Mumbai Indians vs Chennai Super Kings, 1st Mat...",2018-04-07,"Wankhede Stadium,",Mumbai,Chennai Super Kings won by 1 wkt,Mumbai Indians Innings,innings_1,Hardik Pandya,not out,22,20,2,0


In [14]:
df.loc[df.runs,['batsman_name','runs']].max()

batsman_name    Yuvraj Singh
runs                      68
dtype: object

In [17]:
df.shape

(963, 13)

In [19]:
#list down the century makers, their scores and their strike rates

df['strikerate']=(df.runs/df.balls).round(2)

df.loc[df.runs>=100,['batsman_name','runs','balls','strikerate']].sort_values(by='runs',ascending=0)



,batsman_name,runs,balls,strikerate
680,Rishabh Pant (wk),128,63,2.03
959,Shane Watson,117,57,2.05
285,Shane Watson,106,57,1.86
274,Chris Gayle,104,63,1.65
739,Ambati Rayudu,100,62,1.61


In [6]:
#list down the top 5 most consistent batsmen
df2=df.groupby('batsman_name').runs.std().sort_values(ascending=True)
df2.head()

batsman_name
Vinay Kumar        0.000000
Dhawal Kulkarni    0.000000
Sandeep Sharma     0.000000
Umesh Yadav        0.547723
Liam Plunkett      0.707107
Name: runs, dtype: float64

In [21]:
#list down the top 5 most consistent batsmen
df1=df.groupby('batsman_name').runs.std().sort_values(ascending=False)
con1=df1.dropna().sort_values(ascending=True)
con1.head()

batsman_name
Vinay Kumar          0.000000
Dhawal Kulkarni      0.000000
Sandeep Sharma       0.000000
Umesh Yadav          0.547723
Mustafizur Rahman    0.707107
Name: runs, dtype: float64

In [22]:
# what are the total number of duck dismissals?
df0=df[(df.runs==0)&(df.dismissal_mode!='not out')]
df_ducks=df0.loc[df0.runs==0,['batsman_name','dismissal_mode','runs']].sort_values(by='runs')
print('Total number of duck dismisals:',df_ducks.runs.count())
print("Below id the list of all the duck dismissals:")
df_ducks.head()


Total number of duck dismisals: 68
Below id the list of all the duck dismissals:


,batsman_name,dismissal_mode,runs
1,Evin Lewis,lbw b D Chahar,0
437,Rishabh Pant (wk),c Karthik b A Russell,0
447,Shivam Mavi,b A Mishra,0
455,Dwayne Bravo,c Markande b McClenaghan,0
474,Ben Stokes,b Y Pathan,0


In [23]:
# list top 5 players by contribution of boundaries to total scores
df['boundary_runs'] = df['sixes']*6 + df['fours']*4
grp = df.groupby('batsman_name')
df1 = grp['runs','boundary_runs'].agg({'runs':np.sum,'boundary_runs':np.sum})
df1
df1['boundary_contrib'] = (df1.boundary_runs/df1.runs)*100
df1.sort_values(by='runs',ascending=0).head()

,runs,boundary_runs,boundary_contrib
batsman_name,,,
Kane Williamson (c),735,420,57.142857
Rishabh Pant (wk),712,460,64.606742
Lokesh Rahul (wk),710,428,60.281690
Ambati Rayudu,602,416,69.102990
Virat Kohli (c),561,326,58.110517


In [24]:
# which location is most batsman friendly
#grp_location = df.groupby('match_location')
df1 = df.groupby('match_location')['runs','boundary_runs'].apply(np.sum)
df1['match_count']=df.groupby('match_location').match_description.nunique()
df1
df1['runs_per_match']=(df1.runs/df1.match_count).round()
df1['boundary_contrib'] = (df1.boundary_runs/df1.runs).round(2)


df1.head()


,runs,boundary_runs,match_count,runs_per_match,boundary_contrib
match_location,,,,,
Bengaluru,2463,1526,7,352.0,0.62
Chennai,395,266,1,395.0,0.67
Delhi,2285,1394,7,326.0,0.61
Hyderabad,2034,1096,7,291.0,0.54
Indore,1248,826,4,312.0,0.66
